In [1]:
# replace_Oppo= {"West":"West indies","Sri":"Sri lanka","South":"South Africa","Hong":"Hong Kong","New":"New Zealand"}
# df["Opposition"]=df["Opposition"].replace(replace_Oppo)

In [2]:
# importing the required libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import re
import seaborn as sns

# Define the URL to scrape
url = "https://stats.espncricinfo.com/ci/engine/player/253802.html?class=11;template=results;type=batting;view=innings"

# Send a request to the URL and parse the HTML content
page = requests.get(url)
soup = BeautifulSoup(page.text)

# Extract relevant data from the HTML using BeautifulSoup
data = soup.find_all('table', class_='engineTable')[3].text.split('\n')
rows = data[22:]

# Extract specific information from the scraped data
runs, mins, BF, fours, sixes, SR, pos, dismissal, inn, ground, date, opposition = ([] for _ in range(12))

for i in range(0, len(rows) - 1, 16):
    runs.append(rows[i])
for i in range(1, len(rows), 16):
    mins.append(rows[i])
for i in range(2, len(rows), 16):
    BF.append(rows[i])
for i in range(3, len(rows), 16):
    fours.append(rows[i])
for i in range(4, len(rows), 16):
    sixes.append(rows[i])
for i in range(5, len(rows), 16):
    SR.append(rows[i])
for i in range(6, len(rows), 16):
    pos.append(rows[i])
for i in range(7, len(rows), 16):
    dismissal.append(rows[i])
for i in range(8, len(rows), 16):
    inn.append(rows[i])
for i in range(11, len(rows), 16):
    ground.append(rows[i])
for i in range(12, len(rows), 16):
    date.append(rows[i])
for i in range(10, len(rows), 16):
    opposition.append(rows[i])

# Create a DataFrame using the extracted information
df = pd.DataFrame({
    "Runs": runs,
    "Mins": mins,
    "BF": BF,  # Balls faced
    "Fours": fours,
    "Sixes": sixes,
    "SR": SR,  # Strike Rate
    "POS": pos,
    "Dismissal": dismissal,
    "INNS": inn,
    "Opposition": opposition,
    "Ground": ground,
    "Date": date
})

# Data cleaning and formatting
columns_to_convert = ['Mins', 'BF', 'Fours', 'Sixes', 'POS', 'INNS']
for col in columns_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

df['SR'] = pd.to_numeric(df['SR'], errors='coerce').astype('Float64')
df['Date'] = pd.to_datetime(df['Date'])
df["Runs"] = df["Runs"].str.replace("*", "")
df["Runs"] = pd.to_numeric(df["Runs"], errors='coerce').astype('Int64')
df["Dismissal"] = df["Dismissal"].replace("-", "Not Played")
df["Dismissal"] = df["Dismissal"].str.title()
df["Dismissal"] = df["Dismissal"].replace('Lbw','LBW')

# Set 'SR' values to 0 if the corresponding 'Runs' is 0
for index, sr_value in df['SR'].items():
    if pd.notna(df['Runs'][index]) and df['Runs'][index] == 0:
        df.at[index, 'SR'] = 0

# Split 'Opposition' column into 'Format' and 'Opposition'
df[['Format', 'Opposition']] = df['Opposition'].str.split('v', expand=True)

# Create a copy of the DataFrame with non-null 'Runs' values
dfc = df.dropna(subset=['Runs'])

# Fill missing values in 'Mins' with the median value (62)
dfc.Mins = df['Mins'].fillna(62)

# Display the DataFrames
print("\nOriginal DataFrame:")
print(df)
print("DataFrame with non-null 'Runs' values:")
print(dfc)


Original DataFrame:
     Runs  Mins  BF  Fours  Sixes      SR  POS Dismissal  INNS     Opposition  \
0      12    33  22      1      0   54.54    2       LBW     1      Sri Lanka   
1      37    82  67      6      0   55.22    2    Caught     2      Sri Lanka   
2      25    40  38      4      0   65.78    1   Run Out     1      Sri Lanka   
3      54    87  66      7      0   81.81    1    Bowled     1      Sri Lanka   
4      31    45  46      3      1   67.39    1       LBW     2      Sri Lanka   
..    ...   ...  ..    ...    ...     ...  ...       ...   ...            ...   
601    54    99  63      4      0   85.71    3    Bowled     1      Australia   
602    38    96  64      5      0   59.37    4    Caught     1   South Africa   
603    76   137  82     12      1   92.68    4    Caught     3   South Africa   
604    46   103  59      6      1   77.96    4    Caught     2   South Africa   
605    12    13  11      2      0  109.09    4    Caught     4   South Africa   

      

C:\Users\MUJAHID\AppData\Local\Temp\ipykernel_11832\3904956673.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.Mins = df['Mins'].fillna(62)


In [3]:
df.isna().sum()

Runs          28
Mins          95
BF            28
Fours         28
Sixes         28
SR            28
POS           28
Dismissal      0
INNS           7
Opposition     0
Ground         0
Date           0
Format         0
dtype: int64

In [4]:
dfc.isna().sum()

Runs          0
Mins          0
BF            0
Fours         0
Sixes         0
SR            0
POS           0
Dismissal     0
INNS          0
Opposition    0
Ground        0
Date          0
Format        0
dtype: int64

In [5]:
dfc.head(50)

,Runs,Mins,BF,Fours,Sixes,SR,POS,Dismissal,INNS,Opposition,Ground,Date,Format
0,12,33,22,1,0,54.54,2,LBW,1,Sri Lanka,Dambulla,2008-08-18,ODI
1,37,82,67,6,0,55.22,2,Caught,2,Sri Lanka,Dambulla,2008-08-20,ODI
2,25,40,38,4,0,65.78,1,Run Out,1,Sri Lanka,Colombo (RPS),2008-08-24,ODI
3,54,87,66,7,0,81.81,1,Bowled,1,Sri Lanka,Colombo (RPS),2008-08-27,ODI
4,31,45,46,3,1,67.39,1,LBW,2,Sri Lanka,Colombo (RPS),2008-08-29,ODI
5,2,6,2,0,0,100.0,7,Not Out,1,Sri Lanka,Colombo (RPS),2009-09-14,ODI
6,16,28,24,1,0,66.66,4,Caught,2,Pakistan,Centurion,2009-09-26,ODI
8,79,121,104,9,2,75.96,4,Not Out,2,West Indies,Johannesburg,2009-09-30,ODI
9,30,53,41,3,0,73.17,4,Caught,2,Australia,Vadodara,2009-10-25,ODI
10,10,32,16,1,0,62.5,3,Caught,2,Australia,Mohali,2009-11-02,ODI


In [6]:
dfc.SR.mean()

77.71498269896195

In [7]:
df.SR.mean()

77.71498269896193

In [10]:
centuries = dfc[dfc.Runs >= 100]


In [12]:
centuries['SR'].mean()

96.615